In [1]:
#loads the 4-class multiclass model and does inference on given audio data

#4-class CNN
    #0: miscellanious
    #1: gunshot
    #2: fireworks
    #3: glassbreak

#imports
import glob
import os
from os.path import isdir, join
from pathlib import Path

import numpy as np
from scipy.fftpack import fft
from scipy import signal
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls

import pandas as pd
import librosa
import re
from sklearn.model_selection import KFold

import seaborn as sns
import IPython.display as ipd
import librosa.display

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Input, layers, optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import csv


In [8]:
#auc custom metric
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

#other things
gunshot_frequency_threshold = 0.25
sample_rate = 22050
sample_rate_per_two_seconds = 44100
input_shape = (sample_rate_per_two_seconds, 1)

In [6]:
#paths

model_path = "/Users/laurenogden/Downloads/gunshot_sound_model_multiclass_4.h5"

#a_labels = "/home/lauogden/data/gunshot_augmented_sound_labels_multiclass.npy"
#a_samples = "//home/lauogden/data/gunshot_augmented_sound_samples_multiclass.npy"

#b_labels = "/home/lauogden/data/gunshot_augmented_sound_labels_multiclass.npy"
#b_samples = "/home/lauogden/data/gunshot_augmented_sound_samples_multiclass.npy"


In [4]:
#load the model the old fashioned way

#Model parameters
drop_out_rate = 0.1
learning_rate = 0.001
number_of_epochs = 50
number_of_classes = 4
batch_size = 32
optimizer = optimizers.Adam(learning_rate, learning_rate / 100)
input_tensor = Input(shape=input_shape)
metrics = [auc, "accuracy"]

#architecture
x = layers.Conv1D(16, 9, activation="relu", padding="same")(input_tensor)
x = layers.Conv1D(16, 9, activation="relu", padding="same")(x)
x = layers.MaxPool1D(16)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.MaxPool1D(4)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.MaxPool1D(4)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dropout(rate=(drop_out_rate * 2))(x) # Increasing drop-out rate here to prevent overfitting

x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(1028, activation="relu")(x)
output_tensor = layers.Dense(number_of_classes, activation="softmax")(x)

model = tf.keras.Model(input_tensor, output_tensor)
model.compile(optimizer=optimizer, loss=keras.losses.binary_crossentropy, metrics=metrics)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [7]:
#load weights

model.load_weights(model_path)

In [9]:
#load samples and labels
samples = np.load("/Users/laurenogden/gunshot_detection/raspberry_pi/recordings/reformed_microphone_data_pi.npy")


In [12]:
ipd.Audio(samples.reshape(44100), rate=22050)

In [ ]:
#to categorical, reshape
#label_np_1 = np.array(keras.utils.to_categorical(label_np, 4))
#sample_np = np.array(scont).reshape(-1, sampling_rate_per_two_seconds, 1)

In [13]:
#predict
#4-class CNN
    #0: miscellanious
    #1: gunshot
    #2: fireworks
    #3: glassbreak
    
#this will give you a 0-4 number for what it thinks it is
print(np.argmax(model.predict(samples),axis=1))

print(model.predict(samples))

[0]
[[9.7054201e-01 4.0273867e-03 2.4468226e-02 9.6232450e-04]]


In [ ]:
#find differences
    #if diff = 0, that means the model predicted it correctly
diff = label_np - predictions


In [ ]:
#get the indices of the samples it got wrong
indexes = []
for i in range(len(diff)):
    if diff[i]!=0:
        indexes.append(i)


In [ ]:
#for each index, write the wav file and put it in the text file
for ind in indexes:
    if label_np[ind]==1:
        direc = results_guns
    else:
        direc = results_others
    filepath = direc+"/"+str(ind)+".wav"
    print(filepath)
    librosa.output.write_wav(filepath,scont[ind],sr)
    file.write("filename: " + str(ind) + ".wav\n")
    file.write("    actual label: " + str(label_np[ind])+ "\n")
    file.write("    predicted label: " + str(predictions[ind])+ "\n")